In [130]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/vehicle-sales-data/car_prices.csv


# Setup
Importing data

In [131]:
vehicle_location = '../input/vehicle-sales-data/car_prices.csv'
vehicle_data = pd.read_csv(vehicle_location)

# Showing table top 5 rows

In [132]:
vehicle_data.transmission[vehicle_data.transmission != 'automatic']

35           NaN
44           NaN
62        manual
89           NaN
102       manual
           ...  
558820       NaN
558825    manual
558829       NaN
558830       NaN
558832       NaN
Name: transmission, Length: 82922, dtype: object

# Formating data

Spliting `saledate` to `sale_day`, `sale_month`, `sale_year` 

In [133]:
vehicle_data.dtypes

year              int64
make             object
model            object
trim             object
body             object
transmission     object
vin              object
state            object
condition       float64
odometer        float64
color            object
interior         object
seller           object
mmr             float64
sellingprice    float64
saledate         object
dtype: object

In [134]:
vehicle_data_formated = vehicle_data
vehicle_data_formated['saledate'] = pd.to_datetime(vehicle_data['saledate'], errors = 'coerce')

/tmp/ipykernel_33/813024025.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  vehicle_data_formated['saledate'] = pd.to_datetime(vehicle_data['saledate'], errors = 'coerce')
/tmp/ipykernel_33/813024025.py:2: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  vehicle_data_formated['saledate'] = pd.to_datetime(vehicle_data['saledate'], errors = 'coerce')


## Hecking errors

In [135]:
vehicle_data_formated[vehicle_data['saledate'].isnull()].head()

,year,make,model,trim,body,transmission,vin,state,condition,odometer,color,interior,seller,mmr,sellingprice,saledate
306447,2013,Hyundai,Sonata,GLS,Sedan,automatic,5npeb4ac6dh687932,il,19.0,37254.0,gray,gray,kfl llc,NaN,NaN,NaN
406524,2013,Chrysler,300,Base,Sedan,automatic,2c3ccaag9dh723146,il,27.0,44208.0,silver,black,kfl llc,NaN,NaN,NaN
408161,2015,Volkswagen,Jetta,SE PZEV w/Connectivity,Navitgation,Sedan,automatic,3vwd17aj4fm201708,NaN,46.0,4802,silver,gray,NaN,13200.0,NaT
417835,2015,Volkswagen,Jetta,SE PZEV w/Connectivity,Navitgation,Sedan,automatic,3vwd17aj2fm258506,NaN,1.0,9410,white,gray,NaN,13300.0,NaT
421289,2015,Volkswagen,Jetta,SE PZEV w/Connectivity,Navitgation,Sedan,automatic,3vwd17aj3fm276741,NaN,46.0,1167,blue,black,NaN,13200.0,NaT


My thought about `NaN` value is that these cars have not been sold yet. \
I don't delete records because I can use them in the future to analyze the least selling cars. \
\
I try to change cells because they are mixed between columnss.

In [136]:
#function to find transmission in columns 
def find_transmission(row):
    for record in row:
        if record == 'automatic' or record == 'manual':
            return record
    return row['transmission']
#apply function and changing data
vehicle_data_formated['transmission'] = vehicle_data.apply(find_transmission, axis=1)
#checking what left
vehicle_data_formated[(vehicle_data_formated['transmission'] != 'automatic') & (vehicle_data_formated['transmission'] != 'manual') & (
vehicle_data_formated['transmission'].isna() !=True)]

,year,make,model,trim,body,transmission,vin,state,condition,odometer,color,interior,seller,mmr,sellingprice,saledate
461612,2015,Volkswagen,Jetta,SE PZEV w/Connectivity,Navitgation,Sedan,NaN,3vwd17aj3fm259017,NaN,46.0,2711,white,black,NaN,14250.0,NaT
505299,2015,Volkswagen,Jetta,SE PZEV w/Connectivity,Navitgation,sedan,NaN,3vwd17aj7fm222388,NaN,36.0,20379,silver,black,NaN,13600.0,NaT
529009,2015,Volkswagen,Jetta,SE PZEV w/Connectivity,Navitgation,sedan,NaN,3vwd17aj8fm298895,NaN,2.0,2817,red,black,NaN,13750.0,NaT
551222,2015,Volkswagen,Jetta,SE PZEV w/Connectivity,Navitgation,sedan,NaN,3vwd17aj8fm239622,NaN,2.0,9562,silver,black,NaN,13200.0,NaT


In [137]:
def find_vin(row):
    for record in row:
        try:
            if len(record) == 17:
                return record
        except:
            pass
    return row['vin']
vehicle_data_formated['vin'] = vehicle_data.apply(find_vin, axis = 1)

In [138]:
def find_state(row):
    for record in row:
        try:
            if len(record) == 2 and record.islower():
                return record
        except:
            pass
    return row['state']
vehicle_data_formated['state'] = vehicle_data.apply(find_state, axis = 1)

In [139]:
vehicle_data_formated

,year,make,model,trim,body,transmission,vin,state,condition,odometer,color,interior,seller,mmr,sellingprice,saledate
0,2015,Kia,Sorento,LX,SUV,automatic,5xyktca69fg566472,ca,5.0,16639.0,white,black,kia motors america inc,20500.0,21500.0,2014-12-16 12:30:00+08:00
1,2015,Kia,Sorento,LX,SUV,automatic,5xyktca69fg561319,ca,5.0,9393.0,white,beige,kia motors america inc,20800.0,21500.0,2014-12-16 12:30:00+08:00
2,2014,BMW,3 Series,328i SULEV,Sedan,automatic,wba3c1c51ek116351,ca,45.0,1331.0,gray,black,financial services remarketing (lease),31900.0,30000.0,2015-01-15 04:30:00+08:00
3,2015,Volvo,S60,T5,Sedan,automatic,yv1612tb4f1310987,ca,41.0,14282.0,white,black,volvo na rep/world omni,27500.0,27750.0,2015-01-29 04:30:00+08:00
4,2014,BMW,6 Series Gran Coupe,650i,Sedan,automatic,wba6b2c57ed129731,ca,43.0,2641.0,gray,black,financial services remarketing (lease),66000.0,67000.0,2014-12-18 12:30:00+08:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
558832,2015,Kia,K900,Luxury,Sedan,NaN,knalw4d4xf6019304,in,45.0,18255.0,silver,black,avis corporation,35300.0,33000.0,2015-07-09 07:00:00+07:00
558833,2012,Ram,2500,Power Wagon,Crew Cab,automatic,3c6td5et6cg112407,wa,5.0,54393.0,white,black,i -5 uhlmann rv,30200.0,30800.0,2015-07-08 09:30:00+07:00
558834,2012,BMW,X5,xDrive35d,SUV,automatic,5uxzw0c58cl668465,ca,48.0,50561.0,black,black,financial services remarketing (lease),29800.0,34000.0,2015-07-08 09:30:00+07:00
558835,2015,Nissan,Altima,2.5 S,sedan,automatic,1n4al3ap0fc216050,ga,38.0,16658.0,white,black,enterprise vehicle exchange / tra / rental / t...,15100.0,11100.0,2015-07-09 06:45:00+07:00
